In [6]:
import pandas as pd

In [9]:
import pandas as pd

# Read the Excel file
file_path = 'ExlOutput/AWSML_Q10.xlsx'  # Replace with your actual file path
df = pd.read_excel(file_path) 

# Define the base SQL template for posts table
sql_post_template = """
INSERT INTO wp_jrf23a6sz5_posts
(`post_author`, `post_date`, `post_date_gmt`, `post_content`, `post_title`, 
 `post_excerpt`, `post_status`, `comment_status`, `ping_status`, `post_password`, 
 `post_name`, `to_ping`, `pinged`, `post_modified`, `post_modified_gmt`, 
 `post_content_filtered`, `post_parent`, `guid`, `menu_order`, `post_type`, 
 `post_mime_type`, `comment_count`) 
VALUES 
(2, 
 NOW(), 
 NOW(), 
 '{post_content}', 
 '{post_title}', 
 '', 
 'Publish', 
 'closed', 
 '', 
 '', 
 'AWS ML Question-{id}', 
 '', 
 '', 
 NOW(), 
 NOW(), 
 '', 
 0, 
 'http://corporateschool.local/?post_type=lp_question&p={id}', 
 0, 
 'lp_question', 
 '', 
 0);
"""

# Define the base SQL template for answers table
sql_answer_template = """
INSERT INTO wp_jrf23a6sz5_learnpress_question_answers
(`question_id`, `title`, `value`, `order`, `is_true`) 
VALUES 
({question_id}, '{title}', '{value}', {order}, {is_true});
"""

# Function to generate the SQL insert script for both tables
def generate_sql_script(df):
    sql_post_scripts = []
    sql_answer_scripts = []
    sql_postmeta_script = []
    
    for index, row in df.iterrows():
        post_content = row['Question Text (HTML)']
        post_title = row['Question no.']
        post_id = index + 1  # Assuming auto-increment starts at 1, adjust if needed
        
        # Generate SQL for posts table
        post_sql = sql_post_template.format(post_content=post_content.replace("'", "''"), 
                                            post_title=post_title.replace("'", "''"), 
                                            id=post_id)
        sql_post_scripts.append(post_sql)
        
        # Get the correct answers as a list from the 'Correct Answer(s)' column
        correct_answers = str(row['Correct answer(s)']).split(',')
        correct_answers = [answer.strip() for answer in correct_answers]  # Clean up spaces

        # Generate SQL for each answer based on Option columns
        for answer_order in range(1, 6):  # Assuming maximum 5 options (Option 1 to Option 5)
            option_col = f'Option {answer_order}'
            
            if option_col in row and pd.notna(row[option_col]):
                answer_text = row[option_col]
                
                # Check if the current option (e.g., "B", "D") is listed as a correct answer
                is_true = 1 if chr(64 + answer_order) in correct_answers else 0
                
                # Generate SQL for answers table
                answer_sql = sql_answer_template.format(question_id= "@lastQuestionId",
                                                        title=answer_text.replace("'", "''"),
                                                        value=answer_text.replace("'", "''"),  # Assuming title and value are the same
                                                        order=answer_order,
                                                        is_true=is_true)
                sql_answer_scripts.append(answer_sql)


    
    return sql_post_scripts, sql_answer_scripts


SQL insert scripts for posts have been written to SqlOutput/insert_posts_statements.sql


In [11]:

# Generate SQL scripts
sql_post_scripts, sql_answer_scripts = generate_sql_script(df)

# Save the SQL scripts to files
output_file_posts = 'SqlOutput/insert_posts_statements.sql'
# output_file_answers = 'SqlOutput\insert_answers_statements.sql'

with open(output_file_posts, 'w', encoding='utf-8') as f_posts:
    for script in sql_post_scripts:
        f_posts.write(script + '\n')
        f_posts.write("set @lastQuestionId = LAST_Insert_ID();" + '\n')
    for script in sql_answer_scripts:
        f_posts.write(script + '\n')

# with open(output_file_answers, 'w', encoding='utf-8') as f_answers:
#     for script in sql_answer_scripts:
#         f_answers.write(script + '\n')

print(f"SQL insert scripts for posts have been written to {output_file_posts}")
# print(f"SQL insert scripts for answers have been written to {output_file_answers}")


SQL insert scripts for posts have been written to SqlOutput/insert_posts_statements.sql
